In [1]:
import torch
from torch import nn
import transformers
from datasets import load_dataset
import html

In [ ]:
vi_en_dataset = load_dataset(
    "mt_eng_vietnamese", "iwslt2015-vi-en", cache_dir="dataset"
)


def unescape(batch):
    batch["vi"] = html.unescape(batch["translation"]["vi"])
    batch["en"] = html.unescape(batch["translation"]["en"])
    return batch


del vi_en_dataset["validation"]
vi_en_dataset = vi_en_dataset.map(unescape, remove_columns="translation")
vi_en_dataset.save_to_disk("dataset/processed")

In [78]:
from datasets import load_from_disk

vi_en_dataset = load_from_disk("dataset/processed")